### Importing the Libraries

In [5]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Using TensorFlow backend.


In [4]:
tf.__version__ 

'2.3.1'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [6]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [10]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [8]:
training_set = train_datagen.flow_from_directory('../dataSet/trainingData',                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 12845 images belonging to 27 classes.


In [11]:
test_set = test_datagen.flow_from_directory('../dataSet/testingData',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 4268 images belonging to 27 classes.


### Part 2 - Building the CNN

#### Initializing the CNN

In [12]:
classifier = tf.keras.models.Sequential()

#### Step 1 - Convolution

In [13]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#### Step 2 - Pooling

In [14]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Adding a second convolutional layer

In [15]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Step 3 - Flattening

In [16]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [17]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [18]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [19]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               4194432   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0

In [21]:
classifier.fit(training_set,
                  epochs = 5,
                  validation_data = test_set)

Epoch 1/5
1285/1285 [==============================] - 291s 227ms/step - loss: 1.8812 - accuracy: 0.4090 - val_loss: 0.4482 - val_accuracy: 0.8732
Epoch 2/5
1285/1285 [==============================] - 195s 152ms/step - loss: 0.9843 - accuracy: 0.6678 - val_loss: 0.2235 - val_accuracy: 0.9323
Epoch 3/5
1285/1285 [==============================] - 190s 148ms/step - loss: 0.7293 - accuracy: 0.7480 - val_loss: 0.1038 - val_accuracy: 0.9688
Epoch 4/5
1285/1285 [==============================] - 215s 168ms/step - loss: 0.6005 - accuracy: 0.7960 - val_loss: 0.0605 - val_accuracy: 0.9866
Epoch 5/5
1285/1285 [==============================] - 207s 161ms/step - loss: 0.5017 - accuracy: 0.8324 - val_loss: 0.0518 - val_accuracy: 0.9873


#### Saving the Model

In [22]:
model_json = classifier.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new.h5')
print('Weights saved')

Model Saved
Weights saved
